In [1]:
import os
import sys
sys.path.append("../GradVisV2/toolbox")
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')
import matplotlib
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
import time
import colorama
from matplotlib import animation
import Visualization as vis
import nn_model
import trajectory_plots as tplot

In [2]:
#Random seed initialization
seed = 1234
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
class Neural_net_parameters():
    
    def __init__(self):

        self.Dict_initializers = {"GlorotNormal"  : "GlorotNormal",
                                  "GlorotUniform" : "GlorotUniform",
                                  "Ones"          : "Ones",
                                  "RandomNormal"  : "RandomNormal",
                                  "RandomUniform" : "RandomUniform",
                                  "Zeros"         : "Zeros"}
        
        self.Dict_activations = {"relu"         : tf.nn.relu,
                                 "leaky_relu"   : tf.nn.leaky_relu,
                                 "sigmoid"      : tf.nn.sigmoid,
                                 "tanh"         : tf.nn.tanh,
                                 "elu"          : tf.nn.elu,
                                 "hard sigmoid" : tf.keras.activations.hard_sigmoid,
                                 "linear"       : tf.keras.activations.linear,
                                 "selu"         : tf.keras.activations.selu,
                                 "softmax"      : tf.keras.activations.softmax}
        
        self.Dict_optimizers = {"Adadelta" : tf.keras.optimizers.Adadelta(learning_rate = 0.001, rho = 0.95, epsilon = 1e-07),
                                "Adagrad"  : tf.keras.optimizers.Adagrad(learning_rate = 0.001, initial_accumulator_value = 0.1, epsilon = 1e-07),
                                "Adam"     : tf.keras.optimizers.Adam(learning_rate = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-07, amsgrad = False),
                                "Adamax"   : tf.keras.optimizers.Nadam(learning_rate = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-07),
                                "Nadam"    : tf.keras.optimizers.Nadam(learning_rate = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-07),
                                "RMSprop"  : tf.keras.optimizers.RMSprop(learning_rate = 0.001, rho = 0.9, momentum = 0.0, epsilon = 1e-07, centered = False),
                                "SGD"      : tf.keras.optimizers.SGD(learning_rate = 0.01, momentum = 0.0, nesterov = False)}
        
        self.Dict = {"initializer" : self.Dict_initializers, "activation" : self.Dict_activations, "optimizer" : self.Dict_optimizers}



class ODEsolver():
    
    def __init__(self, x, initial_condition, epochs, architecture, initializer, activation, optimizer, prediction_save, weights_save):
        """
        x : training domain (ex: x = np.linspace(0, 1, 100))
        initial_condition : initial condition including x0 and y0 (ex: initial_condition = (x0 = 0, y0 = 1))
        architecture : number of nodes in hidden layers (ex: architecture = [10, 10])
        initializer : weight initializer (ex: 'GlorotNormal')
        activation : activation function (ex: tf.nn.sigmoid)
        optimizer : minimization optimizer including parameters (ex: tf.keras.optimizers.Adam(learning_rate = 0.001, beta_1 = 0.5, beta_2 = 0.5, epsilon = 1e-07))
        prediciton_save : bool to save predicitons at each epoch during training (ex: prediction_save = False)
        weights_save : bool to save the weights at each epoch (ex: weights_save = True)
        """
        colorama.init()
        self.GREEN = colorama.Fore.GREEN
        self.RESET = colorama.Fore.RESET
        tf.keras.backend.set_floatx('float64')
        self.x = x
        self.initial_condition = initial_condition
        self.n = len(self.x)
        self.epochs = epochs
        self.architecture = architecture
        self.initializer = initializer
        self.activation = activation
        self.optimizer = optimizer
        self.neural_net = self.build_model()#self.neural_net_model(show = True)
        self.neural_net.summary()
        
        #Compile the model
        x = self.x
        x = tf.convert_to_tensor(x)
        x = tf.reshape(x, (self.n, 1))
        self.neural_net.compile(loss = self.custom_cost(x), optimizer = self.optimizer, experimental_run_tf_function = False)
        print("------- Model compiled -------")

        #Raise an exception is both prediction_save and weights_save are True
        if prediction_save and weights_save:
        	raise Exception("Both prediciton_save and weights_save are set to True.")
        if prediction_save:
            self.predictions = []

        
        
        
    def build_model(self):
        """
        Builds a customized neural network model.
        """
        architecture = self.architecture
        initializer = self.initializer
        activation = self.activation
        
        nb_hidden_layers = len(architecture)
        input_tensor = tf.keras.layers.Input(shape = (1,))
        hidden_layers = []

        if nb_hidden_layers >= 1:
            hidden_layer = tf.keras.layers.Dense(architecture[0], kernel_initializer= initializer, bias_initializer='zeros',activation = activation)(input_tensor)
            hidden_layers.append(hidden_layer)
            for i in range(1, nb_hidden_layers):
                hidden_layer = tf.keras.layers.Dense(architecture[i], kernel_initializer= initializer, bias_initializer='zeros',activation = activation)(hidden_layers[i-1])
                hidden_layers.append(hidden_layer)
            output_layer = tf.keras.layers.Dense(1, kernel_initializer= initializer, bias_initializer = 'zeros', activation = tf.identity)(hidden_layers[-1])
        else:
            output_layer = tf.keras.layers.Dense(1, kernel_initializer= initializer, bias_initializer = 'zeros', activation = tf.identity)(input_tensor)
        
        model = tf.keras.Model(inputs = input_tensor, outputs = output_layer)
        return model
    
    
    @tf.function
    def NN_output(self, x):
        """
        x : must be of shape = (?, 1)
        Returns the output of the neural net
        """
        y = self.neural_net(x)
        return y
    

    def y_gradients(self, x):
        """
        Computes the gradient of y.
        """
        with tf.GradientTape() as tape1:
            tape1.watch(x)
            with tf.GradientTape() as tape2:
                tape2.watch(x)
                y = self.NN_output(x)
            dy_dx = tape2.gradient(y, x)
        d2y_dx2 = tape1.gradient(dy_dx,x)
        return y, dy_dx, d2y_dx2

    
    def differential_cost(self, x):
        """
        Defines the differential cost function for one neural network
        input.
        """
        y, dy_dx, d2y_dx2 = self.y_gradients(x)

        #----------------------------------------------
        #------------DIFFERENTIAL-EQUATION-------------
        #----------------------------------------------
        differential_equation = dy_dx + y - tf.exp(-x)*tf.cos(x)
        #----------------------------------------------
        #----------------------------------------------
        #----------------------------------------------

        return tf.square(differential_equation)


    def custom_cost(self, x):
        """
        Defines the cost function for a batch.
        """
        x0 = self.initial_condition[0]
        y0 = self.initial_condition[1]
        def loss(y_true, y_pred):
            differential_cost_term = tf.math.reduce_sum(self.differential_cost(x))
            boundary_cost_term = tf.square(self.NN_output(np.asarray([[x0]]))[0][0] - y0)
            return differential_cost_term/self.n + boundary_cost_term
        return loss
    
    
    def train(self):
        """
        neural_net : The built neural network returned by self.neural_net_model
        Trains the model according to x.
        """
        x = self.x
        x = tf.convert_to_tensor(x)
        x = tf.reshape(x, (self.n, 1))
        neural_net = self.neural_net
        
        #Train and save the predicitons
        if prediction_save:
            predictions = self.predictions

            #Define custom callback for predictions during training
            class PredictionCallback(tf.keras.callbacks.Callback):
                def on_epoch_end(self, epoch, logs={}):
                    y_predict = neural_net.predict(x)
                    predictions.append(y_predict)
                    print('Prediction saved at epoch: {}'.format(epoch))

            start_time = time.time()
            history = neural_net.fit(x = x, y = x, batch_size = self.n, epochs = self.epochs, callbacks = [PredictionCallback()])
            print(f"{self.GREEN}---   %s seconds ---  " % (time.time() - start_time))
            print(f"{self.RESET}")
            predictions = tf.reshape(predictions, (self.epochs, self.n))
        
        #Train and save the weights
        if weights_save:
           # if epoch is in filename then should save all
            checkpoint_filepath = './data/weights_test/weights.{epoch:d}.hdf5'
            model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_filepath,
            save_weights_only=True,
            mode='auto',
            save_best_only=False, 
            save_freq = 'epoch')
            start_time = time.time()
            history = neural_net.fit(x = x, y = x, batch_size = self.n, epochs = self.epochs, callbacks=[model_checkpoint_callback])
            print(f"{self.GREEN}---   %s seconds ---  " % (time.time() - start_time))
            print(f"{self.RESET}")

        #Train without any saving
        else:
           

            start_time = time.time()
            history = neural_net.fit(x = x, y = x, batch_size = self.n, epochs = self.epochs)
            print(f"{self.GREEN}---   %s seconds ---  " % (time.time() - start_time))
            print(f"{self.RESET}")
            
        return history
    
    
    def get_loss(self, history):
        """
        history : history of the training procedure returned by self.train
        Returns epochs and loss
        """
        epochs = history.epoch
        loss = history.history["loss"]
        return epochs, loss
    
    
    def predict(self, x_predict):
        """
        x_predict : domain of prediction (ex: x_predict = np.linspace(0, 1, 100))
        """
        domain_length = len(x_predict)
        x_predict = tf.convert_to_tensor(x_predict)
        x = tf.reshape(x_predict, (domain_length, 1))
        y_predict = self.neural_net.predict(x_predict)
        return y_predict


    def relative_error(self, y_predict, y_exact):
        """
        y_predict : array of predicted solution
        y_exact : array of exact solution
        Returns the relative error of the neural network solution
        given the exact solution.
        """
        if len(y_exact) != len(y_predict):
            raise Exception("y_predict and y_exact do not have the same shape.")
        relative_error = np.abs(y_exact - np.reshape(y_predict, (100)))/np.abs(y_exact)
        return relative_error


    def mean_relative_error(self, y_predict, y_exact):
    	"""
    	y_predict : array of predicted solution
        y_exact : array of exact solution
        Returns the single mean relative error value of 
        the neural network solution given the exact solution.
    	"""
    	relative_error = self.relative_error(y_predict, y_exact)
    	relative_error = relative_error[relative_error < 1E100]
    	return np.mean(relative_error)


    def absolute_error(self, y_predict, y_exact):
        """
        y_predict : array of predicted solution
        y_exact : array of exact solution
        Returns the mean absolute error of the neural network solution
        given the exact solution.
        """
        if len(y_exact) != len(y_predict):
            raise Exception("y_predict and y_exact do not have the same shape.")
        absolute_error = np.abs(y_exact - np.reshape(y_predict, (100)))
        return absolute_error
    
    
    def get_predictions(self):
        """
        Returns the neural net predictions at each epoch 
        """
        if not prediction_save:
            raise Exception("The predictions have not been saved.")
        else:
            return self.predictions
        
        
    def training_animation(self, y_exact, y_predict, epoch, loss):
        """
        Plot the training animation including the exact solution, 
        the neural network solution and the cost function as functions of epochs.
        This function needs the model to be trained and requires the outputs of get_loss.
        """
        #Position of loss function (can be "upper right" or "lower right")
        position = "lower right"
        
        if not prediction_save:
            raise Exception("The predictions have not been saved.")
        fig, ax = plt.subplots()
        ax1 = plt.axes()
        if position == "lower right":
            ax2 = fig.add_axes([0.58, 0.2, 0.3, 0.2])
        if position == "upper right":
            ax2 = fig.add_axes([0.58, 0.65, 0.3, 0.2])

        frames = []

        x = self.x
        predictions = self.predictions

        x_loss = epoch
        y_loss = loss

        ax1.plot(x, y_exact, "C1", label = "Exact solution")
        ax1.legend(loc = 'upper left')
        ax1.set_xlim(min(x), max(x))
        ax1.set_ylim(min(y_predict) - 0.1, max(y_predict) + 0.1)
        ax1.set_xlabel("$x$", fontsize = 15)
        ax1.set_ylabel("$\hat{f}$", fontsize = 15)

        ax2.set_xlabel("Epochs")
        ax2.set_ylabel("Loss")
        ax2.semilogy(x_loss, y_loss, color = "w", linewidth = 1.0)

        x_loss_points = []
        y_loss_points = []

        for i in range(self.epochs):
            y = predictions[i]
            frame1, = ax1.plot(x, y, ".", color = "C0", markersize = 3)
            x_loss_points.append(x_loss[i])
            y_loss_points.append(y_loss[i])
            frame2, = ax2.semilogy(x_loss_points, y_loss_points, color = "C0")
            frames.append([frame1, frame2])

        ani = animation.ArtistAnimation(fig, frames, interval = 10, blit = True)
        Writer = animation.writers['ffmpeg']
        writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)
        ani.save('./data/weights_test/animation.mp4', writer=writer)
        #plt.show()
        
        
    def plot_solution(self, x_predict, y_predict, y_exact):
        """
        Plot the neural net solution with the exact solution
        including the relative error.
        """
        fig = plt.figure()

        #Exact and numerical solution
        axe1 = fig.add_axes([0.17, 0.35, 0.75, 0.6])
        axe1.set_ylabel("$\hat{f}$", fontsize = 15)
        axe1.set_xticks([])
        #Relative error
        axe2 = fig.add_axes([0.17, 0.1, 0.75, 0.25])
        axe2.set_ylabel("Relative \n error", fontsize = 15)
        axe2.set_xlabel("$x$", fontsize = 15)
        axe2.set_yscale('log')

        axe1.plot(x_predict, y_exact, color = "C1", label = "Exact solution")
        axe1.plot(x_predict, y_predict, ".", color = "C0", label = "Neural network solution", markersize = 3)
        axe1.legend()

        axe2.plot(x_predict, self.relative_error(y_predict, y_exact), color = "C0")
        plt.show()
        
    
        
        

In [4]:


if __name__ == "__main__":

    #--------------------------------------------------------------------
    #-----------------PARAMETER-INITIALIZATION---------------------------
    #--------------------------------------------------------------------

    #Import the neural network parameters
    nnp = Neural_net_parameters()
    Dict = nnp.Dict

    #Training domain
    x = np.linspace(0, 6, 600)
    #Initial conditions
    initial_condition = (0, 0)
    #Number of epochs
    epochs = 1000
    #Structure of the neural net (only hidden layers)
    architecture = [10]
    #Initializer used
    initializer = Dict["initializer"]["GlorotNormal"]
    #Activation function used
    activation = Dict["activation"]["sigmoid"]
    #Optimizer used
    optimizer = Dict["optimizer"]["Adam"]
    #Save predictions at each epoch in self.predictions
    prediction_save = False
    weights_save = True


    #--------------------------------------------------------------------
    #------------------MODEL-DEFINITION-AND-TRAINING---------------------
    #--------------------------------------------------------------------

    #Class definition
    solver = ODEsolver(x, initial_condition, epochs, architecture, initializer, activation, optimizer, prediction_save, weights_save)
    #Training
    history = solver.train()
    epoch, loss = solver.get_loss(history)

    #--------------------------------------------------------------------
    #------------------PREDICTION----------------------------------------
    #--------------------------------------------------------------------

    """
    #Plot the exact and the neural net solution
    x_predict = x
    y_predict = solver.predict(x_predict)
    y_exact = np.exp(-x_predict)*np.sin(x_predict)
    solver.plot_solution(x_predict, y_predict, y_exact)
    print(solver.mean_relative_error(y_predict, y_exact))
    """

    #--------------------------------------------------------------------
    #------------------TRAINING-ANIMATION--------------------------------
    #--------------------------------------------------------------------
    x_predict = x
    y_predict = solver.predict(x_predict)
    #solver.training_animation(np.zeros(len(y_predict)), y_predict, epoch, loss)


    #--------------------------------------------------------------------
    #------------------LOSS-SURFACE--------------------------------
    #--------------------------------------------------------------------
    #solver.neural_net.save_weights('./data/minimum_0')
    #USE: model_instance = nn.model.Tensorflow_NNModel(model, loss_func, x, file)
    #nnmodel = nn_model.Tensorflow_NNModel(solver.neural_net, solver.neural_net.loss, solver.x, './data/minimum_0')

    # PLOT LOSS SURFACE
    #USE: vis.visualize(model_instance, loss_func, x, [file], N_mesh, path_to_file, random_dir = False for PCA/True, proz = scaling of coordinates, verbose)
    # NOTE THAT WE NEED [] around the file!!
    #vis.visualize(nnmodel,solver.neural_net.loss,solver.x, ['./data/minimum_0'], 20, './data/example',random_dir = True, proz = 1.5, verbose=True)
    #NOTE BE CAREFUL WITH OVERWRITING AND SAME FILENAMES WHEN RERUNNING, EITHER DELETE THOSE DATA FILES OR CHANGE NAME
    #USE: tplot.plot_loss_2D(file.npz, path_to_save_plot, is_log), same with 3D but have degrees for camera angle
    #tplot.plot_loss_2D('./data/example.npz','./data/plot2d',is_log=False)
    #plot.plot_loss_3D('./data/example.npz','./data/plot3d',is_log=False, degrees = 120)
    #outs = np.load('./data/example.npz', allow_pickle=True)
    #outs = outs["a"]
    # outs stores information of loss and the meshgrid: outs[0][0] is X; outs[0][1] is Y; outs[0][0] is LOSS
    # USE: tplot.plot3D(X,Y,LOSS, HTML_SAVE_FILE)
    #tplot.plot3D(outs[0][0],outs[0][1],outs[0][2], './data/interactive_plot.html')




Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
dense (Dense)                (None, 10)                20        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 31
Trainable params: 31
Non-trainable params: 0
_________________________________________________________________
------- Model compiled -------
Epoch 1/1000
1/1 [==============================] - ETA: 0s - loss: 0.772 - 0s 17ms/step - loss: 0.7729
Epoch 2/1000
1/1 [==============================] - ETA: 0s - loss: 0.756 - 0s 19ms/step - loss: 0.7568
Epoch 3/1000
1/1 [==============================] - ETA: 0s - loss: 0.741 - 0s 16ms/step - loss: 0.7410
Epoch 4/1000
1/1 [==============================] - 

Epoch 72/1000
1/1 [==============================] - ETA: 0s - loss: 0.196 - 0s 6ms/step - loss: 0.1961
Epoch 73/1000
1/1 [==============================] - ETA: 0s - loss: 0.193 - 0s 11ms/step - loss: 0.1936
Epoch 74/1000
1/1 [==============================] - ETA: 0s - loss: 0.191 - 0s 21ms/step - loss: 0.1913
Epoch 75/1000
1/1 [==============================] - ETA: 0s - loss: 0.189 - 0s 19ms/step - loss: 0.1890
Epoch 76/1000
1/1 [==============================] - ETA: 0s - loss: 0.186 - 0s 11ms/step - loss: 0.1868
Epoch 77/1000
1/1 [==============================] - ETA: 0s - loss: 0.184 - 0s 12ms/step - loss: 0.1846
Epoch 78/1000
1/1 [==============================] - ETA: 0s - loss: 0.182 - 0s 8ms/step - loss: 0.1825
Epoch 79/1000
1/1 [==============================] - ETA: 0s - loss: 0.180 - 0s 9ms/step - loss: 0.1804
Epoch 80/1000
1/1 [==============================] - ETA: 0s - loss: 0.178 - 0s 9ms/step - loss: 0.1784
Epoch 81/1000
1/1 [==============================] - ETA: 0

1/1 [==============================] - ETA: 0s - loss: 0.099 - 0s 12ms/step - loss: 0.0994
Epoch 150/1000
1/1 [==============================] - ETA: 0s - loss: 0.098 - 0s 11ms/step - loss: 0.0987
Epoch 151/1000
1/1 [==============================] - ETA: 0s - loss: 0.098 - 0s 12ms/step - loss: 0.0981
Epoch 152/1000
1/1 [==============================] - ETA: 0s - loss: 0.097 - 0s 12ms/step - loss: 0.0974
Epoch 153/1000
1/1 [==============================] - ETA: 0s - loss: 0.096 - 0s 28ms/step - loss: 0.0967
Epoch 154/1000
1/1 [==============================] - ETA: 0s - loss: 0.096 - 0s 14ms/step - loss: 0.0961
Epoch 155/1000
1/1 [==============================] - ETA: 0s - loss: 0.095 - 0s 16ms/step - loss: 0.0955
Epoch 156/1000
1/1 [==============================] - ETA: 0s - loss: 0.094 - 0s 20ms/step - loss: 0.0948
Epoch 157/1000
1/1 [==============================] - ETA: 0s - loss: 0.094 - 0s 19ms/step - loss: 0.0942
Epoch 158/1000
1/1 [==============================] - ETA: 0s

1/1 [==============================] - ETA: 0s - loss: 0.066 - 0s 10ms/step - loss: 0.0667
Epoch 227/1000
1/1 [==============================] - ETA: 0s - loss: 0.066 - 0s 19ms/step - loss: 0.0665
Epoch 228/1000
1/1 [==============================] - ETA: 0s - loss: 0.066 - 0s 8ms/step - loss: 0.0662
Epoch 229/1000
1/1 [==============================] - ETA: 0s - loss: 0.066 - 0s 15ms/step - loss: 0.0660
Epoch 230/1000
1/1 [==============================] - ETA: 0s - loss: 0.065 - 0s 11ms/step - loss: 0.0658
Epoch 231/1000
1/1 [==============================] - ETA: 0s - loss: 0.065 - 0s 11ms/step - loss: 0.0656
Epoch 232/1000
1/1 [==============================] - ETA: 0s - loss: 0.065 - 0s 11ms/step - loss: 0.0653
Epoch 233/1000
1/1 [==============================] - ETA: 0s - loss: 0.065 - 0s 9ms/step - loss: 0.0651
Epoch 234/1000
1/1 [==============================] - ETA: 0s - loss: 0.064 - 0s 14ms/step - loss: 0.0649
Epoch 235/1000
1/1 [==============================] - ETA: 0s -

1/1 [==============================] - ETA: 0s - loss: 0.056 - 0s 10ms/step - loss: 0.0561
Epoch 304/1000
1/1 [==============================] - ETA: 0s - loss: 0.056 - 0s 11ms/step - loss: 0.0560
Epoch 305/1000
1/1 [==============================] - ETA: 0s - loss: 0.055 - 0s 14ms/step - loss: 0.0559
Epoch 306/1000
1/1 [==============================] - ETA: 0s - loss: 0.055 - 0s 10ms/step - loss: 0.0559
Epoch 307/1000
1/1 [==============================] - ETA: 0s - loss: 0.055 - 0s 11ms/step - loss: 0.0558
Epoch 308/1000
1/1 [==============================] - ETA: 0s - loss: 0.055 - 0s 7ms/step - loss: 0.0557
Epoch 309/1000
1/1 [==============================] - ETA: 0s - loss: 0.055 - 0s 12ms/step - loss: 0.0557
Epoch 310/1000
1/1 [==============================] - ETA: 0s - loss: 0.055 - 0s 9ms/step - loss: 0.0556
Epoch 311/1000
1/1 [==============================] - ETA: 0s - loss: 0.055 - 0s 8ms/step - loss: 0.0555
Epoch 312/1000
1/1 [==============================] - ETA: 0s - 

1/1 [==============================] - ETA: 0s - loss: 0.052 - 0s 12ms/step - loss: 0.0526
Epoch 381/1000
1/1 [==============================] - ETA: 0s - loss: 0.052 - 0s 11ms/step - loss: 0.0526
Epoch 382/1000
1/1 [==============================] - ETA: 0s - loss: 0.052 - 0s 17ms/step - loss: 0.0526
Epoch 383/1000
1/1 [==============================] - ETA: 0s - loss: 0.052 - 0s 9ms/step - loss: 0.0525
Epoch 384/1000
1/1 [==============================] - ETA: 0s - loss: 0.052 - 0s 17ms/step - loss: 0.0525
Epoch 385/1000
1/1 [==============================] - ETA: 0s - loss: 0.052 - 0s 12ms/step - loss: 0.0525
Epoch 386/1000
1/1 [==============================] - ETA: 0s - loss: 0.052 - 0s 13ms/step - loss: 0.0525
Epoch 387/1000
1/1 [==============================] - ETA: 0s - loss: 0.052 - 0s 24ms/step - loss: 0.0524
Epoch 388/1000
1/1 [==============================] - ETA: 0s - loss: 0.052 - 0s 18ms/step - loss: 0.0524
Epoch 389/1000
1/1 [==============================] - ETA: 0s 

KeyboardInterrupt: 

In [5]:
final_state = "./data/weights_test/weights."+str(int(epochs))+"hdf5"

nnmodel = nn_model.Tensorflow_NNModel(solver.neural_net, solver.neural_net.loss, solver.x, final_state)

Build Tensorflow Model ...
Setting up Model ...
Initializing Parameters...
Done.


In [6]:
num_files = epochs
fileindices=np.linspace(1,num_files,num_files,num_files)

In [8]:
filesname = ["./data/weights_test/weights."+str(int(i))+".hdf5" for i in fileindices]

vis.visualize(nnmodel,solver.neural_net.loss,solver.x, filesname, 150, './data/weights_test/example',random_dir = True, proz = 10, verbose=True)

Calculating directions...
Normalizing...
Calculating loss landscape...


  0%|          | 0/150 [00:00<?, ?it/s]

Average Loss: tf.Tensor(141.6238841594348, shape=(), dtype=float64)
Average Loss: tf.Tensor(139.52838389938842, shape=(), dtype=float64)
Average Loss: tf.Tensor(137.44658968158913, shape=(), dtype=float64)
Average Loss: tf.Tensor(135.37890099990005, shape=(), dtype=float64)
Average Loss: tf.Tensor(133.32572678051872, shape=(), dtype=float64)
Average Loss: tf.Tensor(131.28747749601226, shape=(), dtype=float64)
Average Loss: tf.Tensor(129.26457510407, shape=(), dtype=float64)
Average Loss: tf.Tensor(127.25743775773299, shape=(), dtype=float64)
Average Loss: tf.Tensor(125.26649392751156, shape=(), dtype=float64)
Average Loss: tf.Tensor(123.29217649996009, shape=(), dtype=float64)
Average Loss: tf.Tensor(121.33492018322893, shape=(), dtype=float64)
Average Loss: tf.Tensor(119.3951714342275, shape=(), dtype=float64)
Average Loss: tf.Tensor(117.47337512780099, shape=(), dtype=float64)
Average Loss: tf.Tensor(115.5699827652004, shape=(), dtype=float64)
Average Loss: tf.Tensor(113.685444771561

Average Loss: tf.Tensor(4.534506261631031, shape=(), dtype=float64)
Average Loss: tf.Tensor(4.193034502552817, shape=(), dtype=float64)
Average Loss: tf.Tensor(3.865934277303139, shape=(), dtype=float64)
Average Loss: tf.Tensor(3.5531871676041913, shape=(), dtype=float64)
Average Loss: tf.Tensor(3.2547774576395425, shape=(), dtype=float64)
Average Loss: tf.Tensor(2.9706921553147647, shape=(), dtype=float64)
Average Loss: tf.Tensor(2.700920851326784, shape=(), dtype=float64)
Average Loss: tf.Tensor(2.4454561109528434, shape=(), dtype=float64)
Average Loss: tf.Tensor(2.2042928274471647, shape=(), dtype=float64)
Average Loss: tf.Tensor(1.9774279267244066, shape=(), dtype=float64)
Average Loss: tf.Tensor(1.764864301131149, shape=(), dtype=float64)
Average Loss: tf.Tensor(1.566600823240035, shape=(), dtype=float64)
Average Loss: tf.Tensor(1.3826455024599353, shape=(), dtype=float64)
Average Loss: tf.Tensor(1.2130019167233081, shape=(), dtype=float64)
Average Loss: tf.Tensor(1.05768132870307

  1%|          | 1/150 [00:02<05:49,  2.35s/it]

...

Average Loss: tf.Tensor(87.75926395662309, shape=(), dtype=float64)
Average Loss: tf.Tensor(89.44505394500021, shape=(), dtype=float64)
Average Loss: tf.Tensor(91.14716790615202, shape=(), dtype=float64)
Average Loss: tf.Tensor(92.86516953615377, shape=(), dtype=float64)
Average Loss: tf.Tensor(94.59858966995739, shape=(), dtype=float64)
Average Loss: tf.Tensor(96.34693560563633, shape=(), dtype=float64)
Average Loss: tf.Tensor(98.10968342765997, shape=(), dtype=float64)
Average Loss: tf.Tensor(99.88629980399045, shape=(), dtype=float64)
Average Loss: tf.Tensor(101.6762359135725, shape=(), dtype=float64)
Average Loss: tf.Tensor(103.47894501840672, shape=(), dtype=float64)
Average Loss: tf.Tensor(105.29388655341236, shape=(), dtype=float64)
Average Loss: tf.Tensor(107.12050967547042, shape=(), dtype=float64)
Average Loss: tf.Tensor(108.95833814553016, shape=(), dtype=float64)
Average Loss: tf.Tensor(110.80686660738338, shape=(), dtype=float64)
Average Loss: tf.Tensor(112.66568478850783,

 99%|█████████▉| 149/150 [04:35<00:02,  2.19s/it]

Average Loss: tf.Tensor(0.4435007789405983, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.3514722045844988, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.27264054098992924, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.20700020608521338, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.15454631436760952, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.11527480578595171, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.08918247162378029, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.07626710153591285, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.07652757147891044, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.08996396015865023, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.11657767063840024, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.15637157271235486, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.20935011602388287, shape=(), dtype=float64)
Average Loss: tf.Tensor(0.2755195081795185, shape=(), dtype=float64)
Average Loss: tf.Tensor

Average Loss: tf.Tensor(89.2312839694707, shape=(), dtype=float64)
Average Loss: tf.Tensor(90.931740347552, shape=(), dtype=float64)
Average Loss: tf.Tensor(92.6488263310925, shape=(), dtype=float64)
Average Loss: tf.Tensor(94.38212256818787, shape=(), dtype=float64)
Average Loss: tf.Tensor(96.13117480869823, shape=(), dtype=float64)
Average Loss: tf.Tensor(97.89550251823002, shape=(), dtype=float64)
Average Loss: tf.Tensor(99.67459029399447, shape=(), dtype=float64)
Average Loss: tf.Tensor(101.4679091854929, shape=(), dtype=float64)
Average Loss: tf.Tensor(103.27490981645043, shape=(), dtype=float64)
Average Loss: tf.Tensor(105.09503969749221, shape=(), dtype=float64)
Average Loss: tf.Tensor(106.92774681386896, shape=(), dtype=float64)
Average Loss: tf.Tensor(108.77246300922202, shape=(), dtype=float64)
Average Loss: tf.Tensor(110.62868956586303, shape=(), dtype=float64)
Average Loss: tf.Tensor(112.4958928152877, shape=(), dtype=float64)
Average Loss: tf.Tensor(114.37362980625055, sha

100%|██████████| 150/150 [04:38<00:00,  1.85s/it]


In [11]:
outs = np.load('./data/weights_test/example.npz', allow_pickle = True)
flag = outs["b"]
outs = outs["a"]
flag

array(3)

In [14]:
tplot.plot_loss_2D('./data/weights_test/example.npz',contours = 300,'./data/weights_test/plot2d', is_log=False)

SyntaxError: positional argument follows keyword argument (<ipython-input-14-679df2d80a25>, line 1)

In [10]:
tplot.plot_loss_3D('./data/weights_test/example.npz','./data/weights_test/plot3d',height = 45, degrees = 190, is_log=False)

Saving figure ./data/sin_weights_test/plot3d.svg


In [11]:
tplot.plot3D(outs[0][0],outs[0][1],outs[0][2], './data/weights_test/interactive_plot.html')

/usr/lib/python3.8/site-packages/IPython/core/display.py:701: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


True